In [ ]:
import json
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Ellipse
import numpy as np

# Load the data
with open('plot_data.json', 'r') as f:
    plot_data = json.load(f)

# Set up the figure and axis
fig, ax = plt.subplots(figsize=(10, 10))
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)

# Set background color
bg_color = f"#{plot_data['plot_parameters']['background_color']:06x}"
fig.patch.set_facecolor(bg_color)
ax.set_facecolor(bg_color)

# Draw segments (bars)
bar_height = plot_data['plot_parameters']['bar_height_setting']
for segment in plot_data['segments']:
    rect = Rectangle((segment['x'] - segment['width']/2, 0.8),
                     segment['width'], bar_height,
                     facecolor=segment['color'])
    ax.add_patch(rect)

# Draw abstractions (ellipses) and connection lines
for category, abstraction in plot_data['abstractions'].items():
    if abstraction and all(k in abstraction for k in ('x', 'y', 'size', 'color')):
        # Draw ellipse
        ellipse = Ellipse((abstraction['x'], abstraction['y']),
                          0.04 + abstraction['size']/0.9,
                          0.04 + abstraction['size'],
                          facecolor=abstraction['color'])
        ax.add_patch(ellipse)
        
        # Add text
        ax.text(abstraction['x'], abstraction['y'], category,
                ha='center', va='center',
                wrap=True, fontsize=8*abstraction['size'])
        
        # Draw connection lines
        for segment in plot_data['segments']:
            if segment['category'] == category:
                ax.plot([segment['x'], abstraction['x']],
                        [0.802, abstraction['y']],
                        color=segment['color'],
                        linewidth=2*segment['width'])

# Remove axes
ax.axis('off')

# Save the figure
plt.savefig('data_visualization.png', dpi=300, bbox_inches='tight')
plt.close()

print("Visualization saved as 'data_visualization.png'")

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import os
import asyncio

In [10]:
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')

# Create a Service instance with ChromeDriverManager
service = Service(ChromeDriverManager().install())
if "THIRD_PARTY_NOTICES" in service.path:
    service = Service(service.path.replace("THIRD_PARTY_NOTICES.chromedriver", "chromedriver.exe"))

# Initialize the Chrome WebDriver with the specified service and options
driver = webdriver.Chrome(service=service, options=options)

driver.implicitly_wait(10)

try:
    # Go to the page
    # driver.get("https://vyneer.me/vods/")

    driver.get("http://localhost:5173/details?video_id=OqVH_MTBQ6k")

    await asyncio.sleep(5)

    # Now the page is fully loaded, including content loaded via JavaScript 
    html_content = driver.page_source



    # Get the image
    svg_element = driver.find_element(By.CSS_SELECTOR, "svg")

    svg_element.screenshot("recap_image.png")






    # Close the browser
    driver.quit()
    print("Driver closed")

except Exception as e:
    driver.quit()
    print("Driver closed")

Driver closed


In [ ]:
# Initialize the Chrome WebDriver with the specified service and options
driver = webdriver.Chrome(service=service, options=options)

driver.implicitly_wait(10)

# Go to the page
# driver.get("https://vyneer.me/vods/")

driver.get("http://localhost:5173/details?video_id=OqVH_MTBQ6k")

await asyncio.sleep(5)

# Now the page is fully loaded, including content loaded via JavaScript 
html_content = driver.page_source



# Get the image
svg_element = driver.find_element(By.CSS_SELECTOR, "svg")

# svg_element.screenshot("recap_image.png")

In [ ]:
# # Locate the SVG element
# svg_element = driver.find_element(By.CSS_SELECTOR, "svg")

# # Get the bounding box of the SVG element
# location = svg_element.location
# size = svg_element.size
# left = location['x']
# top = location['y']
# width = size['width']
# height = size['height']

# # Capture a full-page screenshot
# driver.save_screenshot("full_page.png")

# # Open the full-page screenshot and crop it to the SVG element
# from PIL import Image

# full_page_screenshot = Image.open("full_page.png")
# svg_screenshot = full_page_screenshot.crop((left, top, left + width, top + height))
# svg_screenshot.save("svg_image.png")

In [11]:
from svgutils.transform import fromstring

In [13]:
driver = webdriver.Chrome(service=service, options=options)

driver.implicitly_wait(10)

# Go to the page
# driver.get("https://vyneer.me/vods/")

driver.get("http://localhost:5173/details?video_id=OqVH_MTBQ6k")

await asyncio.sleep(5)

# Now the page is fully loaded, including content loaded via JavaScript 
html_content = driver.page_source


AttributeError: 'bytes' object has no attribute 'encode'

In [14]:
# Locate the SVG element
svg_element = driver.find_element(By.CSS_SELECTOR, "svg")

# Extract the SVG markup
svg_markup = svg_element.get_attribute('outerHTML')

# Save the SVG markup to a file
with open("image.svg", "w") as file:
    file.write(svg_markup)

# Convert SVG to PNG
svg_data = fromstring(svg_markup.encode('utf-8'))
svg_data.saveas("image.png")

AttributeError: 'bytes' object has no attribute 'encode'

In [15]:
import cairosvg

OSError: no library called "cairo-2" was found
no library called "cairo" was found
no library called "libcairo-2" was found
cannot load library 'libcairo.so.2': error 0x7e.  Additionally, ctypes.util.find_library() did not manage to locate a library called 'libcairo.so.2'
cannot load library 'libcairo.2.dylib': error 0x7e.  Additionally, ctypes.util.find_library() did not manage to locate a library called 'libcairo.2.dylib'
cannot load library 'libcairo-2.dll': error 0x7e.  Additionally, ctypes.util.find_library() did not manage to locate a library called 'libcairo-2.dll'

In [22]:
import asyncio
import aiohttp
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from PIL import Image
import io
import os

async def convert_svg_to_png(svg_path, png_path, width=800, height=600):
    if not os.path.exists(svg_path):
        raise FileNotFoundError(f"SVG file not found: {svg_path}")

    abs_svg_path = os.path.abspath(svg_path)
    svg_url = f"file://{abs_svg_path}"

    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument(f'--window-size={width},{height}')

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    try:
        driver.get(svg_url)

        # Use asyncio.to_thread for potentially blocking Selenium operations
        await asyncio.to_thread(WebDriverWait(driver, 10).until,
            EC.presence_of_element_located((By.TAG_NAME, "svg"))
        )

        svg = await asyncio.to_thread(driver.find_element, By.TAG_NAME, "svg")
        svg_width = await asyncio.to_thread(svg.get_attribute, "width")
        svg_height = await asyncio.to_thread(svg.get_attribute, "height")

        if not svg_width or not svg_height:
            svg_width = await asyncio.to_thread(driver.execute_script, "return document.documentElement.clientWidth;")
            svg_height = await asyncio.to_thread(driver.execute_script, "return document.documentElement.clientHeight;")

        await asyncio.to_thread(driver.set_window_size, int(svg_width), int(svg_height))

        png = await asyncio.to_thread(driver.get_screenshot_as_png)

        img = Image.open(io.BytesIO(png))
        
        img.save(png_path, "PNG")

        print(f"PNG saved to {png_path}")

    finally:
        await asyncio.to_thread(driver.quit)

In [23]:
await convert_svg_to_png(r"D:\DestinyFolder\DestinyRecaps\DestinyRecapsApi\serverproject\Notebooks\Prod_Controllers\image.svg", r"D:\DestinyFolder\DestinyRecaps\DestinyRecapsApi\serverproject\Notebooks\Prod_Controllers\image.png")

PNG saved to D:\DestinyFolder\DestinyRecaps\DestinyRecapsApi\serverproject\Notebooks\Prod_Controllers\image.png
